# Financial Chatbot with LangChain

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline to process 6 text files, answer financial queries, and generate graphs if needed.

## Steps:
1. Setup: Install dependencies and set environment variables.
2. Define LLM Call Function: Create a function for OpenAI GPT-4o-mini API calls.
3. Ingest Text Files: Load text files into a vector store using all-MiniLM-L6-v2 embeddings.
4. Query Matching: Retrieve relevant context from the vector store for the query.
5. LLM Response: Use GPT-4o-mini to answer the query and determine if a graph is needed.
6. Check Graph Requirement: Decide whether to output text or proceed with graph generation.
7. Generate Table for Graph: Create a JSON table for graph data (if needed).
8. Generate Python Code for Graph: Produce code to create and save the graph (if needed).
9. Execute Graph Code: Run the code to save the graph as chart.png (if needed).

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGCHAIN_API_KEY"

In [1]:
# Step 1: Setup
# - Ensure required packages are installed for LangChain, Chroma, and other dependencies.
# - Set up environment variables for OpenAI and LangChain API keys.
# - Ensure matplotlib is installed for graph generation.
# - Create a local directory for text files.

import os

# Note: Ensure these packages are installed locally via pip:
# pip install langchain langchain-community langchain-openai chromadb tiktoken python-dotenv matplotlib sentence-transformers langchain-chroma

# Set environment variables (replace with your actual keys)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGCHAIN_API_KEY"

# Create directory for text files if it doesn't exist
os.makedirs("./data/texts", exist_ok=True)
print("Setup complete. Directory ./data/texts created for text files.")

Setup complete. Directory ./data/texts created for text files.


In [ ]:
# Step 2: Define LLM Call Function
# - Create a function to make API calls to OpenAI's GPT-4o-mini model.
# - Add error handling to catch API failures and print the raw response.
# - Parse the response to handle Markdown-wrapped JSON.
# - This function will be reused in later steps for LLM queries.

import requests
import json
import re

def llm_call(prompt: str) -> str:
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4o-mini",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx, 5xx)
        response_json = response.json()

        # Debug: Print the raw API response
        print("Raw API response:", json.dumps(response_json, indent=2))

        # Check if "choices" exists in the response
        if "choices" not in response_json:
            raise ValueError(f"API response does not contain 'choices'. Response: {response_json}")

        # Extract the content from the response
        content = response_json["choices"][0]["message"]["content"]

        # Remove Markdown code block (e.g., ```json\n{...}\n```) if present
        json_match = re.search(r"```json\n(.*)\n```", content, re.DOTALL)
        if json_match:
            content = json_match.group(1).strip()
        else:
            # If no code block, assume the content is already JSON or plain text
            content = content.strip()

        return content
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response content: {response.text}")
        raise
    except Exception as e:
        print(f"Error during API call: {e}")
        raise

print("LLM call function defined.")

LLM call function defined.


In [19]:
# Step 0: Common LLM Call
from openai import OpenAI

client = OpenAI()

def call_llm(system_prompt, user_prompt):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json"
    }
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [3]:
%pip install "unstructured[all-docs]" unstructured-client watermark python-dotenv pydantic langchain langchain-community langchain_core langchain_openai chromadb

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Fix SSL certificate issue for NLTK downloads on macOS (especially Python 3.13)
import ssl
import certifi

# Override default SSL context
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/arya/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/arya/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [27]:
# ✅ Full Step 1 (Text File Ingestion): Extract → Chunk → Embed into Persistent ChromaDB

import chromadb
from chromadb.utils import embedding_functions
import tiktoken
import os

# --- Reset and Initialize Persistent ChromaDB ---
if 'chroma_client' in globals():
    try:
        chroma_client.reset()
    except:
        pass
    del chroma_client

chroma_client = chromadb.PersistentClient(path="./data/chroma_db")
embedding_func = embedding_functions.DefaultEmbeddingFunction()
collection = chroma_client.get_or_create_collection(name="financial_chunks")

# --- Text File Extraction ---
def extract_from_txt_file(txt_path):
    with open(txt_path, "r", encoding="utf-8") as file:
        raw_text = file.read()
    return [para.strip() for para in raw_text.split("\n\n") if para.strip()]

# --- Token-aware Chunking ---
def chunk_text(text_blocks, max_tokens=300):
    enc = tiktoken.get_encoding("cl100k_base")
    chunks, current_chunk = [], ""
    for block in text_blocks:
        if len(enc.encode(current_chunk + " " + block)) <= max_tokens:
            current_chunk += " " + block
        else:
            chunks.append(current_chunk.strip())
            current_chunk = block
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# --- Store in ChromaDB ---
def store_chunks_in_chromadb(chunks, source_id):
    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk],
            ids=[f"{source_id}_{i}"],
            metadatas=[{"source": source_id}]
        )

In [6]:
# Step 2: Query vector DB and return most relevant chunks
def query_relevant_chunks(query, top_k=3):
    results = collection.query(query_texts=[query], n_results=top_k)
    return results["documents"][0]

In [ ]:
# Step 3: LLM call with query + context → get answer and whether a graph is required
def step3_generate_response_and_check_graph(query, context):
    user_prompt = f"""Question: {query}\nContext:\n{context}\nRespond with JSON: {{"response": "<answer>", "graph_required": true/false}}"""
    r3 = call_llm("", user_prompt)
    print("Step 3 Output:")
    print(r3)
    return r3

In [8]:
# Step 4a: If graph required → generate data table from context + answer
def step4_generate_table(context, response):
    user_prompt = f"""Use the context and response below to extract data as a table (JSON array of rows with headers).\nContext:\n{context}\nResponse:\n{response}\n"""
    r4 = call_llm("", user_prompt)
    print("Step 4a Output (Table):")
    print(r4)
    return r4

In [10]:
# Step 4b: Generate Python code for graph using matplotlib
def step4_generate_graph_code(table, query):
    user_prompt = f"""Use this table:\n{table}\nAnd this query:\n{query}\nWrite Python code using matplotlib to generate and save the graph as 'generated_graph.png'"""
    r5 = call_llm("", user_prompt)
    print("Step 4b Output (Code):")
    print(r5)
    return r5

In [11]:
# Step 4c: Extract and execute code to render graph
import re

def step4_execute_code(llm_code_response):
    code_match = re.search(r"```python(.*?)```", llm_code_response, re.DOTALL)
    if code_match:
        valid_code = code_match.group(1).strip()
        print("Step 4c Executing Graph Code...")
        exec(valid_code)
    else:
        print("No valid code block found in response.")

Ingestion 

In [28]:
# ✅ Define your text file path
txt_path = "/Users/arya/Documents/pfl/data/texts/axis finance annual.txt"   # <-- replace with your actual .txt file
source_id = os.path.basename(txt_path).replace(".txt", "")

# ✅ Run ingestion pipeline
print("📄 Reading text file...")
extracted_blocks = extract_from_txt_file(txt_path)
print(f"✅ Extracted {len(extracted_blocks)} blocks.\n")

print("✂️ Chunking text...")
chunks = chunk_text(extracted_blocks)
print(f"✅ Created {len(chunks)} chunks.\n")

print("📦 Storing chunks into ChromaDB...")
store_chunks_in_chromadb(chunks, source_id)
print(f"✅ Stored chunks for: {source_id}\n")


📄 Reading text file...
✅ Extracted 767 blocks.

✂️ Chunking text...
✅ Created 274 chunks.

📦 Storing chunks into ChromaDB...
✅ Stored chunks for: axis finance annual



Query matching - vectors

In [34]:
# ✅ Step 2: Match query to vector DB chunks

# 🧪 Call with your query
query = "What is the cash and cash equivalent at March 31 2025 and March 31 2024?"

print("🔎 Matching query to ChromaDB vectors...")
matched_context = query_relevant_chunks(query)
print(f"✅ Matched context:\n{matched_context}\n")

🔎 Matching query to ChromaDB vectors...
✅ Matched context:
['For the year ended 31 March 2024', '2023-24  \nGround- 1 (Foreclosure Related) \t1373 \t43% \t29  \nGround - 2 (Collection Related) \t253 \t220% \t25  \nGround - 3 (Refund/Waiver/Discount Related) \t187 \t-43% \t1  \nGround - 4 (EMI Related) \t106 \t-10% \t1  \nGround - 5 (ROI/Tenure Related) \t221 \t-8% \t2  \nOthers \t2775 \t30% \t56  \nTotal \t4915 \t120 \t- [Stamps of Chartered Accountants]  \nB.K. KHAIRE & CO   \nM. KAPADIA & CO  \nChartered\npage_094\nAXIS FINANCE LIMITED\nNotes forming part of financial statements for the year ended March 31, 2025 \n(All amounts are in rupees lakhs, except per share data as stated otherwise)', 'Particulars                                      For the year ended March 31, 2024  \n                                    Stage 1        Stage 2         Stage 3         Total  \nGross carrying amount opening balance    21,87,920.53      28,849.82      13,689.70      22,30,460.05  \nNew assets or

LLM response

In [35]:
# ✅ Step 3: Call LLM to get answer + graph flag
def step3_generate_response_and_check_graph(query, context):
    user_prompt = f"""Question: {query}\nContext:\n{context}\nRespond with JSON: {{"response": "<answer>", "graph_required": true/false}}"""
    r3 = call_llm(system_prompt="", user_prompt=user_prompt)
    print("🧠 Step 3 LLM Output:")
    print(r3)
    return r3

# 🧪 Call the step
step3_output = step3_generate_response_and_check_graph(query, matched_context)

🧠 Step 3 LLM Output:
```json
{"response": "The cash and cash equivalents at March 31, 2025, and March 31, 2024 are not provided in the given context.", "graph_required": false}
```


In [33]:
# ✅ Step 4a: Use LLM to turn text into a tabular JSON format for plotting

# 🧪 Extract `response` field from step 3 output
import json
step3_dict = json.loads(step3_output)
response_text = step3_dict["response"]

# 🧪 Call the table generator
step4_table = step4_generate_table(matched_context, response_text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# ✅ Step 4b: Generate matplotlib code from table + query

# 🧪 Call graph code generator
step4_code = step4_generate_graph_code(step4_table, query)

In [ ]:
# ✅ Step 4c: Extract and execute the generated Python code

# 🧪 Execute graph code
step4_execute_code(step4_code)
print("✅ Graph saved as 'generated_graph.png'")